## Text Classification

BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


FastText is a library for efficient learning of word representations and sentence classification.

The goal of text classification is to assign documents (such as emails, posts, text messages, product reviews, etc...) to one or multiple categories. Such categories can be review scores, spam v.s. non-spam, or the language in which the document was typed. Nowadays, the dominant approach to build such classifiers is machine learning, that is learning classification rules from examples. In order to build such classifiers, we need labeled data, which consists of documents and their corresponding categories (or tags, or labels).

As an example, it builds a classifier which automatically classifies stackexchange questions about cooking into one of several possible tags, such as `pot`, `bowl` or `baking`.

This example notebook is base on the guide from fasttext: https://fasttext.cc/docs/en/supervised-tutorial.html

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fasttext&ea=fasttext-text-classification&dt=fasttext-text-classification)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install -q bentoml "fasttext==0.9.2"

### Prepare data

In [3]:
!curl https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz --output cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  446k  100  446k    0     0   883k      0 --:--:-- --:--:-- --:--:--  883k
x cooking.stackexchange.id
x cooking.stackexchange.txt
x readme.txt


In [4]:
!head cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?
__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?
__label__chocolate American equivalent for British chocolate terms
__label__baking __label__oven __label__convection Fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readymade packed mayonnaise and other sauces


In [5]:
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

### Train model

In [6]:
import fasttext

In [7]:
model = fasttext.train_supervised(input="cooking.train")

### Define and save BentoService

In [14]:
%%writefile text_classification.py

from bentoml import env, artifacts, BentoService, api
from bentoml.frameworks.fasttext import FasttextModelArtifact
from bentoml.adapters import JsonInput

@env(auto_pip_dependencies=True)
@artifacts([FasttextModelArtifact('model')])
class FasttextClassification(BentoService):
    
    @api(input=JsonInput())
    def predict(self, json_list):
        input = [i['text'] for i in json_list]
        result = self.artifacts.model.predict(input)
        # return top result
        prediction_result = [i[0].replace('__label__', '') for i in result[0]]
        return prediction_result

Overwriting text_classification.py


In [15]:
from text_classification import FasttextClassification

svc = FasttextClassification()
svc.pack('model', model)

saved_path = svc.save()

[2020-09-08 17:40:46,255] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.8.6+36.gab94ebfb/bentoml/_version.py
set BentoML-0.8.6+36.gab94ebfb/bentoml/_version.py to '0.8.6+36.gab94ebfb'
[2020-09-08 17:40:51,056] INFO - BentoService bundle 'FasttextClassification:20200908174046_70E6AC' saved to: /Users/bozhaoyu/bentoml/repository/FasttextClassification/20200908174046_70E6AC


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [13]:
!bentoml serve {saved_path}

[2020-05-07 14:06:25,817] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.4. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-07 14:06:25,836] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.4,  but loading from BentoML version 0.7.4+8.g496f22a
[2020-05-07 14:06:25,849] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve FasttextClassification:latest --run-with-ngrok

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Navigate to parent directory of the notebook(so you have reference to the `test.jpg` image), and run the following `curl` command to send the image to REST API server and get a prediction result:

```
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '{"text": "Which baking dish is best to bake a banana bread ?"}'
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [26]:
!bentoml containerize FasttextClassification:latest

[2020-09-08 17:51:57,580] INFO - Getting latest version FasttextClassification:20200908174046_70E6AC
Found Bento: /Users/bozhaoyu/bentoml/repository/FasttextClassification/20200908174046_70E6AC
[2020-09-08 17:51:57,630] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-08 17:51:57,647] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+36.gab94ebfb
Tag not specified, using tag parsed from BentoService: 'fasttextclassification:20200908174046_70E6AC'
Building Docker image fasttextclassification:20200908174046_70E6AC from FasttextClassification:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/FasttextClassification/20200908174046_70E

/Requirement already satisfied: cerberus in /opt/conda/lib/python3.7/site-packages (from bentoml==0.8.6->-r ./requirements.txt (line 2)) (1.3.2)
|Requirement already satisfied: py-zipkin in /opt/conda/lib/python3.7/site-packages (from bentoml==0.8.6->-r ./requirements.txt (line 2)) (0.20.0)
\Requirement already satisfied: Mako in /opt/conda/lib/python3.7/site-packages (from alembic->bentoml==0.8.6->-r ./requirements.txt (line 2)) (1.1.3)
-Requirement already satisfied: jmespath<1.0.0,>=0.7.1 in /opt/conda/lib/python3.7/site-packages (from boto3->bentoml==0.8.6->-r ./requirements.txt (line 2)) (0.10.0)
|  Building wheel for fasttext (setup.py): finished with status 'done'
\  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=4441113 sha256=9e4dca257f27f2bf88fc1900dba10a3cac64bb1cdd62c5868f05e4cae76b5d4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-75ddlgl0/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built f

\Collecting sqlalchemy-utils<0.36.8
-  Downloading SQLAlchemy-Utils-0.36.7.tar.gz (131 kB)
/Requirement already satisfied, skipping upgrade: cerberus in /opt/conda/lib/python3.7/site-packages (from BentoML==0.8.6+36.gab94ebfb) (1.3.2)
|Requirement already satisfied, skipping upgrade: click>=7.0 in /opt/conda/lib/python3.7/site-packages (from BentoML==0.8.6+36.gab94ebfb) (7.1.2)
\Requirement already satisfied, skipping upgrade: grpcio<=1.27.2 in /opt/conda/lib/python3.7/site-packages (from BentoML==0.8.6+36.gab94ebfb) (1.27.2)
-Requirement already satisfied, skipping upgrade: Jinja2>=2.10.1 in /opt/conda/lib/python3.7/site-packages (from flask->BentoML==0.8.6+36.gab94ebfb) (2.11.2)
|  Building wheel for BentoML (PEP 517): finished with status 'done'
  Created wheel for BentoML: filename=BentoML-0.8.6+36.gab94ebfb-py3-none-any.whl size=4712451 sha256=015658c4fada7f2ec4ce508df113796453830a3fe2df0be6986a4a2631549dd5
  Stored in directory: /tmp/pip-ephem-wheel-cache-8blnh_3o/wheels/5

/Installing collected packages: sqlalchemy-utils, BentoML
  Attempting uninstall: sqlalchemy-utils
    Found existing installation: SQLAlchemy-Utils 0.36.8
    Uninstalling SQLAlchemy-Utils-0.36.8:
|      Successfully uninstalled SQLAlchemy-Utils-0.36.8
\  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.8.6
-    Uninstalling BentoML-0.8.6:
|      Successfully uninstalled BentoML-0.8.6
/Successfully installed BentoML-0.8.6+36.gab94ebfb sqlalchemy-utils-0.36.7
- ---> 559bf8aa7c58
Step 10/15 : ENV PORT 5000
 ---> Running in 0904f86ae32f
/ ---> 2fae7d1d07f1
Step 11/15 : EXPOSE $PORT
| ---> Running in 539fe08b13b0
\ ---> 24fee9c226ea
Step 12/15 : COPY docker-entrypoint.sh /usr/local/bin/
- ---> 99b821c976b2
Step 13/15 : RUN chmod +x /usr/local/bin/docker-entrypoint.sh
/ ---> Running in 5ca041bad772
\ ---> eb94d93fd115
Step 14/15 : ENTRYPOINT [ "docker-entrypoint.sh" ]
 ---> Running in e885966e0c53
- ---> 410a2523aa79
Step 15/15 : CMD ["bentoml", "serve

In [ ]:
!docker run --rm -p5000:5000 fasttextclassification:20200908174046_70E6AC

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [ ]:
from bentoml import load

svc = load(saved_path)

print(svc.predict([{"text": "which baking dish is the best?"}]))

## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [25]:
!bentoml run FasttextClassification:latest predict --input='{"text": "Which baking dish is best to bake a banana bread ?"}'

[2020-09-08 17:48:59,442] INFO - Getting latest version FasttextClassification:20200908174046_70E6AC
[2020-09-08 17:48:59,489] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-08 17:48:59,507] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+36.gab94ebfb
([['__label__baking']], [array([0.06232639], dtype=float32)])
['baking']
"baking"


# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

